## Quickstart: $J/\Psi \rightarrow \gamma \pi^0 \pi^0$ decay

In this quickstart example it is shown how to use ComPWA via the python interface. The workflow is:

1. Create a model for the decay.
2. Generate a Monte Carlo data sample (hit & miss) using this model. 
3. Perform a fit on the data sample using the Minuit2 interface.
4. Visualize the data and the fit result!

Let's go!

First we `import` the necessary expert system module parts

In [ ]:
from pycompwa.expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML)

# just a little function to print the intermediate states
def print_intermediate_states(solutions):
    from pycompwa.expertsystem.topology.graph import (
        get_intermediate_state_edges)
    print("intermediate states:")
    intermediate_states = set()
    for g in solutions:
        edge_id = get_intermediate_state_edges(g)[0]
        intermediate_states.add(g.edge_props[edge_id]['@Name'])
    print(intermediate_states)

### Step 1: Creating the Decay Model

#### 1.1. Define problem set

First we define the boundary conditions of our physics problem, such as

- initial state
- final state
- formalism type
- ...

Pass all of that information to the `StateTransitionManager`, which is the main user interface class of the ComPWA expert system

In [ ]:
initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma", [-1, 1]), ("pi0", [0]), ("pi0", [0])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

#### 1.2. Preparation
Create all topology graphs using the isobar model (two-body decays).

Also initialize the graphs with the initial and final state. Remember that each interaction node defines their own set of conservation laws. The `StateTransitionManager` (STM) defines three interaction types:

| Interaction | Strength |
| --- | --- |
| strong | $60$ |
| electromagnetic (EM) | $1$ |
| weak | $10^{-4}$ |

Be default all three are used in the preparation stage. `prepare_graphs()` of the STM generates graphs with all possible combinations of interaction nodes. An overall interaction strength is assigned to each graph, and they are grouped according to this strength.

In [ ]:
graph_interaction_settings_groups = tbd_manager.prepare_graphs()

#### 1.3. Finding solutions
If you are happy with the automatic settings generated by the StateTransitionManager, just go directly to the solving!

In [ ]:
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Ok, now we have a lot of solutions that are actually heavily supressed (involve two weak decays). In general you can modify the dictionary return by `prepare_graphs()` directly.

The STM also comes with a functionality to globally choose the allowed interaction types (`set_allowed_interaction_types()`). Go ahead and **disable** the **EM** and **weak** interaction!

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)
print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Huh, what happened here? Actually, since a $\gamma$ **particle** appears in one of the interaction nodes, the expert system knows that this node must involve **EM interaction**! Because the node can be an effective interaction, the weak interaction cannot be excluded, as it contains only a subset of conservation laws.

Since only the strong interaction was supposed to be used, this gives a warning and automatically corrects the mistake.

Once the EM interaction is included, this warning disappears. However be aware that the EM interaction is now available globally. Hence, now there might be solutions in which both nodes are electromagnetic.

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Great! Now we selected the solutions that are contribution the strongest. However, be aware that there are more effects that can suppress certain decays. For example branching ratios. In this example $J/\Psi$ can decay into $\pi^0 + \rho^0$ or $\pi^0 + \omega$.

| decay | branching ratio |
| --- | --- |
| $$\omega \rightarrow \gamma+\pi^0$$ | 0.0828 |
| $$\rho^0 \rightarrow \gamma+\pi^0$$ | 0.0006 |

Unfortunately the $\rho^0$ decays mainly into $\pi+\pi$, not $\gamma+\pi^0$. Hence it is suppressed. This information is currently not known to the expert system.
But it is possible to hand the expert system a list of allowed intermediate states.

In [ ]:
# particles are found by name comparison; so i.e. f2 will find all f2's and f all f's independent of their spin
tbd_manager.allowed_intermediate_particles = ['f']

(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Now we have selected all amplitudes that involve **f** states. The appearing warnings notify the user, that the solution space is only partial. For certain lines in the graph, no suitable particle was found (since only f state were allowed).

At this point we are all set to generate some data using this amplitude model!

In [ ]:
xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

### Step 2: Creating a data sample

In this section we will use the amplitude model created above to generate a data sample via hit & miss Monte Carlo.

Using this amplitude model in the c++ side of ComPWA is simple. The `create_intensity_and_kinematics()` helper function builds an Intensity and Kinematics object, as specified in the xml file. These can be used to generate our data sample.

In [ ]:
import pycompwa.ui as pwa

intensity, kinematics = pwa.create_intensity_and_kinematics('model.xml')

generator = pwa.RootGenerator(kinematics.get_particle_state_transition_kinematics_info())

random_generator = pwa.RootUniformRealGenerator(12345)

datasample = pwa.generate(5000, kinematics, generator, intensity, random_generator)

### Step 3: Fitting

An **Intensity** object behaves just like a mathematical function that takes set of data as an argument and returns a list of intensities (real numbers). To perform a fit create an estimator of your choice, which generally needs
- an `Intensity` instance
- a `DataSet` (to which the intensity is fitted)
- optionally a phase space `DataSet` (which is used to normalize the Intensity)

A phase space sample can be generated via the `generate_phsp()` function. The data samples can be converted to `DataSet` using the `convert_events_to_dataset()` function.

In [ ]:
phspsample = pwa.generate_phsp(100000, generator, random_generator)

dataset = pwa.convert_events_to_dataset(datasample, kinematics)
phspdataset = pwa.convert_events_to_dataset(phspsample, kinematics)

Now an estimator can be created. Remember that it indicates how well a set of model parameters describes a given data set best. In this example an unbinned log likelihood estimator is used. It performs the calculations with the FunctionTree backend. In this example the intensity is not normalized. Therefore the additional phase space dataset `phspdataset` is supplied, so that the estimator handles the normalization for us.

In [ ]:
estimator, initial_parameters = pwa.create_unbinned_log_likelihood_function_tree_estimator(intensity, 
                                                                                           dataset,
                                                                                           phspdataset)

Notice that you not only receive a estimator object, but also a list of fit parameters (`FitParameterList`). This list of fit parameters are the settings which initialize the optimizer later on. They contain the following info:
- the initial values of the parameters
- fix parameters
- define boundaries
- define errors, which can give certain optimizers hints on the stepsize

These fit parameters are initialized with the values stated in the xml file or default values. But can be changed easily, like fixing certain parameters.

In [ ]:
from pprint import pprint
pprint(initial_parameters)
print("this parameter is initially not fixed: ", initial_parameters[1])
initial_parameters[1].is_fixed = True
print("and now it is fixed: ", initial_parameters[1])

To make the fit a bit more interesting, we modify one of the parameters to a different initial value then the true value.

In [ ]:
print(initial_parameters[6])
initial_parameters[6].value = 2.0
print("after:",initial_parameters[6])

Now it's time to start up a set up a fit, which is quite simple. Just create an optimizer instance of your choice, here Minuit2 (`MinuitIF`), and call the `optimize()` function to start the fitting process.

In [ ]:
minuitif = pwa.MinuitIF()
result = minuitif.optimize(estimator, initial_parameters)

In [ ]:
result.fit_duration_in_seconds

Let's check if the our fit parameters are "close to" the true values.

In [ ]:
print("this should be close to 1.0 again: ", 
      result.final_parameters[6].value, 
      " +- ", 
      result.final_parameters[6].error)

**Important**: Note that the intensity instance still needs to be notified about this optimal set of parameters. They can be applied with the `updateParametersFrom()` function.

In [ ]:
intensity.updateParametersFrom(result.final_parameters)

### Step 4: Visualization
Let's go ahead and make a Dalitz plot of the data sample and our fit result. ComPWA's pycompwa package ships with a little plotting module to help you generate some common plots using matplotlib. Before we can utilize this, the data has to be extended.

Since our model only includes one specific decay topology the `HelicityKinematics` class only generates the kinematic variables needed to evaluate the Intensity. In order to make a Dalitz plot, also the kinematic variables from the other subsystems are needed. They can be created with `create_all_subsystems()`. However the conversion from the event samples have to be performed again!

In [ ]:
kinematics.create_all_subsystems()
dataset = pwa.convert_events_to_dataset(datasample, kinematics)
phspdataset = pwa.convert_events_to_dataset(phspsample, kinematics)

There are two variants to create plots from data:

#### Variant 1: From a ROOT file

The ROOT file can either be generated directly via c++ code (`RootPlotData`), or you can use the `create_rootplotdata()` helper function of the python UI. So let's create a file

In [ ]:
pwa.create_rootplotdata("rootplot.root", kinematics, data_sample=dataset, 
                        phsp_sample=phspdataset, intensity=intensity)

Now the `rootplotdatareader` submodule of `plotting` can be used to read in the data.

In [ ]:
from pycompwa.plotting.rootplotdatareader import open_compwa_plot_data

plot_data = open_compwa_plot_data("rootplot.root")

The returned data structure can directly be fed to `make_dalitz_plots()`

In [ ]:
from pycompwa.plotting import make_dalitz_plots
make_dalitz_plots(plot_data, ["mSq_3_4_vs_2", "mSq_2_4_vs_3"], bins=50)

#### Variant 2: By direct data access

The ComPWA python interface has two helper functions `create_data_array()` `create_fitresult_array()` that allow the datapoins to be exported as two-dimensional lists. These can be used to create a numpy record arrays. Then the **plotting** module is able to create common plots like the Dalitz plot.

In [ ]:
# Plotting
from pycompwa.plotting import (
    make_dalitz_plots, PlotData, create_nprecord,
    make_difference_plot_2d
)

# use the direct data point access
var_names, dataarray = pwa.create_data_array(dataset)
data_record = create_nprecord(var_names, dataarray)

fitres_var_names, fitres_dataarray = pwa.create_fitresult_array(intensity, phspdataset)
fitresult_record = create_nprecord(fitres_var_names, fitres_dataarray)

plot_data = PlotData(data_record=data_record, fit_result_data_record=fitresult_record)
#plot_data.particle_id_to_name_mapping = data_points.get_finalstate_id_to_name_mapping()

# plot a 2d histogram
make_dalitz_plots(plot_data, ["mSq_3_4_vs_2", "mSq_2_4_vs_3"], bins=50)

To make the differences more clearly visible we can create a 2d difference plot

In [ ]:
make_difference_plot_2d(plot_data, ["mSq_3_4_vs_2", "mSq_2_4_vs_3"], bins=50)

That's it. You can check some of the other examples to learn about more detailed features of ComPWA.

And give us feedback or contribute ;)!